<h2> Read in the Data Frame </h2>

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import json


In [3]:
kickstarter_df = pd.read_csv("Kickstarter001.csv")

In [4]:
#Test Run for a sample data set

kickstarter_df = kickstarter_df.head(5)

In [5]:
def extract_creator_id(json_obj):
    try:
        temp=json.loads(json_obj)
    except:
        return None
    return temp['id']
    
creator_id = kickstarter_df['creator'].apply(extract_creator_id)

In [6]:
def extract_reward_urls(json_obj):
    temp = json.loads(json_obj)
    return (temp['web']['rewards'])
    
reward_urls=kickstarter_df['urls'].apply(extract_reward_urls)

<h2> Runtime is pretty high for pull_reward_details (>60 mins for 3800 records) </h2>

In [7]:
def pull_reward_details(url_list):
    
    reward_tier = list()
    #Read the urls and extract details on rewarding tiers
    for i in url_list:
        #print(i)
        response=requests.get(i)
        if response.status_code!=200:
            print("Error pulling data from {}".format(i))
            continue
        reward_desc=bs(response.content,'lxml')
        a=reward_desc.find('ol').find_all('li')
        reward_tier.append(len(a))
    
    return (reward_tier)
        
reward_tier = pull_reward_details(reward_urls)

In [8]:
a = pd.DataFrame({"Project_Id":kickstarter_df['id'],"Creator_Id":creator_id,"Reward_Tiers":reward_tier,"Rewards_Url":reward_urls})

In [9]:
a

,Project_Id,Creator_Id,Reward_Tiers,Rewards_Url
0,418514738,41326882,4,https://www.kickstarter.com/projects/41326882/...
1,831413682,889915542,6,https://www.kickstarter.com/projects/889915542...
2,345681431,384013732,1,https://www.kickstarter.com/projects/384013732...
3,1072644982,945877241,5,https://www.kickstarter.com/projects/orrico/cu...
4,1989538515,1176168309,4,https://www.kickstarter.com/projects/117616830...


<h2> Plotting word clouds for descriptions </h2>